In [1]:
%reload_kedro
catalog = context.catalog

2021-08-24 04:01:49,547 - root - INFO - Registered hooks from 2 installed plugin(s): kedro-mlflow-0.7.3
2021-08-24 04:01:49,633 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-08-24 04:01:49,935 - root - INFO - ** Kedro project Spotify Recommendations
2021-08-24 04:01:49,936 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-08-24 04:01:49,944 - root - INFO - Registered line magic `run_viz`


In [2]:
cleaned_mpd_playlists = catalog.load("cleaned_mpd_playlists")

2021-08-24 04:01:49,966 - kedro.io.data_catalog - INFO - Loading data from `cleaned_mpd_playlists` (ParquetDataSet)...


In [3]:
import dask.dataframe as dd
import dask.distributed as d_dist
from dask.dataframe.core import repartition
from dask.distributed import Client

In [4]:
client = d_dist.client._get_global_client() or Client()

In [5]:
pids_to_tids = cleaned_mpd_playlists[["pid", "track_ids"]].set_index("pid")["track_ids"]

In [6]:
from sklearn.model_selection import train_test_split
#We do not care for order mattering here as of now
#There is no user information so our task will focus on recommending songs to playlists
#  with expansion into optimizing ordering or playlists and then further playlist continuation where sequence matters

In [7]:
split_pids_to_tids = pids_to_tids.map(train_test_split).compute().to_frame().reset_index()

In [8]:
import pandas as pd

In [9]:
split_pids_to_tids[["train_track_ids", "test_track_ids"]] = pd.DataFrame(split_pids_to_tids["track_ids"].tolist(), index= split_pids_to_tids.index)

In [10]:
# train_mpd_pids_to_tids = split_pids_to_tids[["pid", "train_track_ids"]]
# test_mpd_pids_to_tids = split_pids_to_tids[["pid", "test_track_ids"]]

In [11]:
catalog.save("split_mpd_pids_to_tids", split_mpd_pids_to_tids)
# catalog.save("train_mpd_pids_to_tids", train_mpd_pids_to_tids)
# catalog.save("test_mpd_pids_to_tids", test_mpd_pids_to_tids)

NameError: name 'split_mpd_pids_to_tids' is not defined